In [1]:
# Setup
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
from shapely.geometry import Polygon,Point

In [2]:
# Shamelessly lifted function from https://gist.github.com/Sklavit/e05f0b61cb12ac781c93442fbea4fb55

def voronoi_finite_polygons_2d(vor, radius=None):
    """
    Reconstruct infinite voronoi regions in a 2D diagram to finite
    regions.
    Parameters
    ----------
    vor : Voronoi
        Input diagram
    radius : float, optional
        Distance to 'points at infinity'.
    Returns
    -------
    regions : list of tuples
        Indices of vertices in each revised Voronoi regions.
    vertices : list of tuples
        Coordinates for revised Voronoi vertices. Same as coordinates
        of input vertices, with 'points at infinity' appended to the
        end.
    """

    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()*2

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            t = vor.points[p2] - vor.points[p1] # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)

In [3]:
def produce_bounded_regions_padding(df_of_points, x_col='x', y_col='y', padding=0.05):
    # Get (potentially infinite) voronoi cells
    vor = Voronoi(df_of_points[[x_col, y_col]].as_matrix())

    # Constrain regions to be finite (but still potentially huge)
    constrained_regions, constrained_vertices = voronoi_finite_polygons_2d(vor)

    # Convert to shapely polygons
    constrained_polygons = [Polygon([constrained_vertices[k] for k in r]) for r in constrained_regions]

    # Associate our original points (by index) with the constrained regions
    indexed_constrained_polygons = []
    for idx, row in df_of_points.iterrows():
        for cpoly in constrained_polygons:
            if cpoly.contains(Point(row[x_col], row[y_col])):
                indexed_constrained_polygons += [[idx, cpoly]]

    # Create geopandas frame
    df_constrained = gpd.GeoDataFrame({'geometry': [data[1] for data in indexed_constrained_polygons],
                                       'original_index': [data[0] for data in indexed_constrained_polygons]
                                       })

    # Create bounding box as a second geopandas frame
    l = min(df_of_points[x_col])
    r = max(df_of_points[x_col])
    d = min(df_of_points[y_col])
    u = max(df_of_points[y_col])
    width = r - l
    height = u - d
    pad_l = l - padding * width
    pad_r = r + padding * width
    pad_d = d - padding * height
    pad_u = u + padding * height
    boundary_box = Polygon([(pad_l, pad_d), (pad_r, pad_d), (pad_r, pad_u), (pad_l, pad_u)])
    df_bounding = gpd.GeoDataFrame({'geometry': [boundary_box]})

    # Apply bounding box to constrained polygons
    df_bounded = gpd.overlay(df_constrained, df_bounding, how='intersection')

    # Glue back on any additional data values
    df_bounded = df_bounded.merge(df_of_points, how='left', left_on='original_index', right_index=True)

    return df_bounded

In [4]:
def build_tableau_id(row):
    # Hardcoded as we'll definitely create these columns below
    return str(row['original_index'])+'-'+row['geom_type']


def produce_tableau_points_and_polys(bounded_df, data_cols, x_col='x', y_col='y'):
    # For original points, a 'polygon' definition with a single path vertex
    point_columns = list(data_cols)
    point_columns += ['original_index', x_col, y_col]
    point_df = bounded_df[point_columns].copy()
    point_df['geom_type'] = 'point'
    point_df['path_index'] = 1

    # For voronoi cells, a polygon path (avoiding repeated point)
    unpacked_coords = []
    for idx, row in bounded_df.iterrows():
        vertex_count = 1
        cell_boundary = row['geometry'].boundary.coords
        for k in range(len(cell_boundary) - 1):  # -1 to prevent repeat vertex
            unpacked_coords += [[row['original_index'], vertex_count, cell_boundary[k][0], cell_boundary[k][1]]]
            vertex_count += 1

    # Convert to dataframe and add tableau labelling
    poly_df = pd.DataFrame(unpacked_coords, columns=['original_index', 'path_index', x_col, y_col])
    poly_df['geom_type'] = 'polygon'

    # Reattach additional data values
    poly_columns = list(data_cols)
    poly_columns += ['original_index']
    data_df = bounded_df[poly_columns].copy()
    poly_df = poly_df.merge(data_df, how='left', on='original_index')

    # Glue the two sets together
    combined_df = pd.concat([point_df, poly_df])
    
    # Create a tableau_id for detail shelf, that distinguishes points and polygons
    combined_df['tableau_index']=combined_df.apply(lambda row: build_tableau_id(row),axis=1)

    return combined_df

In [5]:
# Get the data set and review
air_quality_data = pd.read_csv('no2-diffusion-tube-data-2016-annual-mean.csv',sep=';')
air_quality_data.head()

,Site,Site ID,X,Y,Latitude,Longitude,LocationType,Pollutant,In AQMA?,Distance to kerb of nearest main road,rec_kerb,Tube_Height,2016 Data Capture,ugm3_2016
0,Gloucester Road,21.0,359027,175300,51.475168,-2.591336,Roadside,NO2,Yes,1.0,20.0,2.8,100.0,50.219591
1,Stokes Croft,22.0,359109,173886,51.462460,-2.589992,Roadside,NO2,Yes,2.0,0.0,2.5,100.0,54.377747
2,Third Way,16.0,352287,178698,51.505189,-2.688835,Roadside,NO2,No,2.0,98.0,2.7,100.0,35.707056
3,Jacobs Wells road near Hotwells roundabout,155.0,357838,172713,51.451820,-2.608146,Roadside,NO2,Yes,2.0,20.0,3.2,91.0,43.137786
4,City Road/Ashley Road lamppost No 16,426.0,359517,174153,51.464890,-2.584150,Roadside,NO2,Yes,2.0,5.0,2.4,100.0,34.216260


In [6]:
# Process (note that longitude is X and latitude is Y!)
aqd = produce_bounded_regions_padding(air_quality_data,'Longitude','Latitude')

In [7]:
# Convert for export
tableau_out = produce_tableau_points_and_polys(aqd, ['Site ID','ugm3_2016','Site'], 'Longitude', 'Latitude')

In [8]:
# Review
tableau_out.head()

,Latitude,Longitude,Site,Site ID,geom_type,original_index,path_index,ugm3_2016,tableau_index
0,51.478063,-2.535216,Fishponds Road,464.0,point,36,1,36.910929,36-point
1,51.475910,-2.553477,Eastville Park,81.0,point,41,1,16.998391,41-point
2,51.488324,-2.582145,No.67 Filton Avenue on wall facing Muller Road,493.0,point,13,1,41.518066,13-point
3,51.493533,-2.586878,On green fence surrounding Western Power subst...,495.0,point,102,1,25.816329,102-point
4,51.494501,-2.656897,Ferndown Close,13.0,point,47,1,18.508268,47-point


In [9]:
# Looks suitable, so push to csv
tableau_out.to_csv("Voronoi_results.csv",index=False)